In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.models import ResNet50
from src.utils.common import UnCallBack
from src.utils.evaluate import makeEval
%matplotlib inline

In [2]:
def commonTrainingProperties(test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = ResNet50()
    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    eval_callback = makeEval(test_loader, loss_fn, device)

    return {"model":model, "optimizer":optimizer, "loss_fn":loss_fn, "eval":eval_callback, "device":device}


In [3]:
%load_ext autoreload
%aimport src.utils.common
%aimport src.utils.evaluate
%aimport src.train_schaul
%autoreload 1

In [ ]:
from src.utils.data_loaders import train_val_dataloader, test_dataloader, train_dataloader

train_loader = train_dataloader(batch_size=300)
test_loader = test_dataloader(batch_size=120)

In [ ]:
from src.train_importance_sampling import  train_full_upper_bound

def upperBoundBndLoss(use_loss_estimation = False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model = ResNet50()
    model.to(device)

    loss_fn = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

    callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'train_w_loss', 'val_loss', 'val_acc', 'train_uniform_cnt'])

    eval_callback = makeEval(test_loader, loss_fn, device)

    train_full_upper_bound( model, 
                train_loader, 
                loss_fn, 
                optimizer, 
                n_epochs = 3, 
                eval = eval_callback, 
                callback = callback, 
                presample = 3, 
                tau_th = None,
                use_loss_estimation = use_loss_estimation,
                second_approach = True,
                device= device)


    return callback

callback_upper_bound = upperBoundBndLoss( use_loss_estimation = False)

callback_loss = upperBoundBndLoss( use_loss_estimation = True)


# Schaul training 

In [4]:
from src.utils.data_loaders import test_dataloader, train_dataloader
#train_loader, val_loader = train_val_dataloader(batch_size=200)#128*3

train_loader = train_dataloader(batch_size=80)
test_loader = test_dataloader(batch_size=80)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
from src.utils.data_loaders import train_val_dataloader, test_dataloader

train_loader, test_loader = train_val_dataloader(batch_size=120)#128*3


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
%autoreload 1

from src.train_schaul import train_full_schaul
if torch.cuda.is_available():
    torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet50()
model.to(device)

loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'val_loss', 'val_acc'])


eval_callback = makeEval(test_loader, loss_fn, device)

train_full_schaul(model, 
                  train_loader, 
                  loss_fn, 
                  optimizer, 
                  n_epochs = 50, 
                  eval = eval_callback, 
                  callback = callback, 
                  device = device)

callback.save("schaul")

# Loshchilov training 

In [1]:
from src.utils.data_loaders import test_dataloader, train_dataloader

train_loader = train_dataloader(batch_size=64)
test_loader = test_dataloader(batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from src.train_loshchilov import train_full_loshchilov

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = ResNet50()
model.to(device)
model.device = device

loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'val_loss', 'val_acc'])

eval_callback = makeEval(train_loader, loss_fn, device)

train_full_loshchilov(model, 
                      train_loader, 
                      loss_fn, 
                      optimizer, 
                      n_epochs=40, 
                      eval = eval_callback, 
                      callback=callback, 
                      device = device)

callback.save("loshchilov")